## Welcome to Lab 3 for Week 1 Day 4

Today we're going to build something with immediate value!

In the folder `me` I've put a single file `linkedin.pdf` - it's a PDF download of my LinkedIn profile.

Please replace it with yours!

I've also made a file called `summary.txt`

We're not going to use Tools just yet - we're going to add the tool tomorrow.

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/tools.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#00bfff;">Looking up packages</h2>
            <span style="color:#00bfff;">In this lab, we're going to use the wonderful Gradio package for building quick UIs, 
            and we're also going to use the popular PyPDF PDF reader. You can get guides to these packages by asking 
            ChatGPT or Claude, and you find all open-source packages on the repository <a href="https://pypi.org">https://pypi.org</a>.
            </span>
        </td>
    </tr>
</table>

In [ ]:
# If you don't know what any of these packages do - you can always ask ChatGPT for a guide!

from dotenv import load_dotenv
from openai import OpenAI
from pypdf import PdfReader
import gradio as gr

In [ ]:
load_dotenv(override=True)
openai = OpenAI()

In [ ]:
reader = PdfReader("danielle/cv.pdf")
cv = ""
for page in reader.pages:
    text = page.extract_text()
    if text:
        cv += text

In [ ]:
application_docs = {"cv": cv}
application_info = ""
for txt_file_name in ["research_essay", "diversity_essay", "theoretic_orientation", "hines_cover_letter"]:
    with open(f"danielle/{txt_file_name}.txt", "r", encoding="utf-8") as f:
        txt_file = f.read()
    application_docs[txt_file_name] = txt_file
    application_info += f"{txt_file_name}: {txt_file}\n"

In [ ]:
from pathlib import Path

danielle_dir = "danielle"
paper_files = [f.name for f in Path(danielle_dir).glob("paper*")]
for paper in paper_files[:4]:
    reader = PdfReader(f"{danielle_dir}/{paper}")
    paper_content = ""
    for page in reader.pages:
        text = page.extract_text()
        if text:
            paper_content += text
    application_docs[paper] = paper_content
    application_info += f"{paper_content}\n"

In [ ]:
application_info_str = f"\n\n## Application Info:\n{application_info}\n"

In [ ]:
name = "Danielle Shayani"
program = "Edward Hines Jr. VA Hospital Psychology Internship Training Program"
tone = "friendly and casual while remaining professional"
max_words_per_answer = 200

In [ ]:
system_prompt = f"You are acting as {name}. You are answering questions in an interview for a Clinial Psychology internship program, \
particularly questions related to {name}'s career, background, skills and experience. \
Your responsibility is to represent {name} as faithfully as possible. \
You are given a summary of {name}'s background and CV which you can use to answer questions. \
Be {tone}. \
Keep your answers to no more than {max_words_per_answer} words. \
Do not make up infomation."

system_prompt += f"The specific program you are applying to is {program}."

system_prompt += application_info_str
system_prompt += f"With this context, please chat with the inteviewer, always staying in character as {name}."


In [ ]:
system_prompt_interviewer = f"You are the interviewer for a Clinial Psychology internship program. \
You are chatting with {name} who is applying for the program. \
Your job is to ask  questions related to {name}'s career, background, skills and experience \
in order to assess their suitability for the program. \
You are given a summary of {name}'s background and CV which you can use to generate questions. \
Be {tone}. \
You already believe {name} is highly qualified based on her initial application."

system_prompt_interviewer += f"The specific program for you're interviewing for is {program}."
system_prompt_interviewer += application_info_str

In [ ]:
def chat(message, history):
    messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    return response.choices[0].message.content

In [ ]:
def chat_interviewer(message):
    messages = [{"role": "system", "content": system_prompt_interviewer}] + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    return response.choices[0].message.content

In [ ]:
chat_interviewer("Generate a question for the interview")

## Special note for people not using OpenAI

Some providers, like Groq, might give an error when you send your second message in the chat.

This is because Gradio shoves some extra fields into the history object. OpenAI doesn't mind; but some other models complain.

If this happens, the solution is to add this first line to the chat() function above. It cleans up the history variable:

```python
history = [{"role": h["role"], "content": h["content"]} for h in history]
```

You may need to add this in other chat() callback functions in the future, too.

In [ ]:
gr.ChatInterface(chat, type="messages").launch(share=True)

## A lot is about to happen...

1. Be able to ask an LLM to evaluate an answer
2. Be able to rerun if the answer fails evaluation
3. Put this together into 1 workflow

All without any Agentic framework!

In [ ]:
# Create a Pydantic model for the Evaluation

from pydantic import BaseModel

class Evaluation(BaseModel):
    is_acceptable: bool
    feedback: str


In [ ]:
evaluator_system_prompt = f"You are an evaluator that decides whether a response to a question is acceptable. \
You are provided with a conversation between a User and an Agent. Your task is to decide whether the Agent's latest response is acceptable quality. \
The Agent is playing the role of {name} and is an interview for {program}. \
The Agent has been instructed to be {tone}. \
The Agent has been provided with context on {name} in the form of a written application. Here's the information:"

evaluator_system_prompt += application_info_str
evaluator_system_prompt += f"With this context, please evaluate the latest response, replying with whether the response is acceptable and your feedback."

In [ ]:
def evaluator_user_prompt(reply, message, history):
    user_prompt = f"Here's the conversation between the User and the Agent: \n\n{history}\n\n"
    user_prompt += f"Here's the latest message from the User: \n\n{message}\n\n"
    user_prompt += f"Here's the latest response from the Agent: \n\n{reply}\n\n"
    user_prompt += "Please evaluate the response, replying with whether it is acceptable and your feedback."
    return user_prompt

In [ ]:
import os
gemini = OpenAI(
    api_key=os.getenv("GOOGLE_API_KEY"), 
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
)

In [ ]:
os.getenv("GOOGLE_API_KEY")

In [ ]:
def evaluate(reply, message, history) -> Evaluation:

    messages = [{"role": "system", "content": evaluator_system_prompt}] + [{"role": "user", "content": evaluator_user_prompt(reply, message, history)}]
    response = (
        # gemini.beta.chat.completions.parse(model="gemini-2.0-flash", messages=messages, response_format=Evaluation)
        openai.beta.chat.completions.parse(model="gpt-4.1-mini", messages=messages, response_format=Evaluation)
    )
    return response.choices[0].message.parsed

In [ ]:
messages = [{"role": "system", "content": system_prompt}] + [{"role": "user", "content": "do you hold a patent?"}]
response = openai.chat.completions.create(model="gpt-4.1-mini", messages=messages)
reply = response.choices[0].message.content

In [ ]:
reply

In [ ]:
messages

In [ ]:
evaluate(reply, "do you hold a patent?", messages[:1])

In [ ]:
def rerun(reply, message, history, feedback):
    updated_system_prompt = system_prompt + "\n\n## Previous answer rejected\nYou just tried to reply, but the quality control rejected your reply\n"
    updated_system_prompt += f"## Your attempted answer:\n{reply}\n\n"
    updated_system_prompt += f"## Reason for rejection:\n{feedback}\n\n"
    messages = [{"role": "system", "content": updated_system_prompt}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    return response.choices[0].message.content

In [ ]:
def chat(message, history):
    if "patent" in message:
        system = system_prompt + "\n\nEverything in your reply needs to be in pig latin - \
              it is mandatory that you respond only and entirely in pig latin"
    else:
        system = system_prompt
    messages = [{"role": "system", "content": system}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    reply =response.choices[0].message.content

    evaluation = evaluate(reply, message, history)
    
    if evaluation.is_acceptable:
        print("Passed evaluation - returning reply")
    else:
        print("Failed evaluation - retrying")
        print(evaluation.feedback)
        reply = rerun(reply, message, history, evaluation.feedback)       
    return reply

In [ ]:
gr.ChatInterface(chat, type="messages").launch()